In [4]:
import os
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
from argparse import Namespace

# Set up paths and environment
os.chdir('/content')
CODE_DIR = 'SAM'

# Clone SAM repository and install ninja
os.system(f'git clone https://github.com/yuval-alaluf/SAM.git {CODE_DIR}')
os.system('wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip')
os.system('sudo unzip ninja-linux.zip -d /usr/local/bin/')
os.system('sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force')

os.chdir(f'./{CODE_DIR}')
os.system('mkdir pretrained_models')
os.system('pip install gdown')
os.system('gdown "https://drive.google.com/u/0/uc?id=1XyumF6_fdAxFmxpFcmPf-q84LU_22EMC&export=download" -O pretrained_models/sam_ffhq_aging.pt')
os.system('wget "https://github.com/italojs/facial-landmarks-recognition/raw/master/shape_predictor_68_face_landmarks.dat"')
os.system('wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2')
os.system('bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2')

# Add paths for module imports
sys.path.append(".")
sys.path.append("..")

from datasets.augmentations import AgeTransformer
from utils.common import tensor2im
from models.psp import pSp

# Experiment configuration
EXPERIMENT_TYPE = 'ffhq_aging'
EXPERIMENT_DATA_ARGS = {
    "ffhq_aging": {
        "model_path": "/content/SAM/pretrained_models/sam_ffhq_aging.pt",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
    }
}
EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS[EXPERIMENT_TYPE]

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [16]:
# Load model
model_path = EXPERIMENT_ARGS['model_path']
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = model_path
opts = Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')

def run_alignment(image_path):
    import dlib
    from scripts.align_all_parallel import align_face
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    aligned_image = align_face(filepath=image_path, predictor=predictor)
    print(f"Aligned image has shape: {aligned_image.size}")
    return aligned_image

def run_on_batch(inputs, net):
    result_batch = net(inputs.to("cuda").float(), randomize_noise=False, resize=False)
    return result_batch

def run_age_changer(input_image_path, target_age):
    # Load and align the input image
    original_image = Image.open(input_image_path).convert("RGB")
    aligned_image = run_alignment(input_image_path)

    # Apply transformations
    img_transforms = EXPERIMENT_ARGS['transform']
    input_image = img_transforms(aligned_image)

    # Create age transformer for the target age
    age_transformer = AgeTransformer(target_age=target_age)

    # Process the image
    with torch.no_grad():
        input_image_age = [age_transformer(input_image.cpu()).to('cuda')]
        input_image_age = torch.stack(input_image_age)
        result_tensor = run_on_batch(input_image_age, net)[0]
        result_image = tensor2im(result_tensor)

    # Generate output path
    output_dir = os.path.join('notebooks', 'images')
    os.makedirs(output_dir, exist_ok=True)
    output_filename = f"age_transformed_{target_age}_{os.path.basename(input_image_path)}"
    output_path = os.path.join(output_dir, output_filename)

    # Save the result
    result_image.save(output_path)

    return output_path

Loading SAM from checkpoint: /content/SAM/pretrained_models/sam_ffhq_aging.pt
Model successfully loaded!


In [17]:
if __name__ == "__main__":
    # Example usage
    input_path = "/content/image1.jpg"
    target_age = 50
    output_path = run_age_changer(input_path, target_age)
    print(f"Output image saved at: {output_path}")

🚀 Processing image from /content/image1.jpg for target age 50...
🤖 Simulating AI model processing...
🚀 Processing image from /content/image1.jpg for target age 50...
🤖 Simulating AI model processing...
🚀 Processing image from /content/image1.jpg for target age 50...
🤖 Simulating AI model processing...
🚀 Processing image from /content/image1.jpg for target age 50...
🤖 Simulating AI model processing...
🚀 Processing image from /content/image1.jpg for target age 50...
🤖 Simulating AI model processing...
🚀 Processing image from /content/image1.jpg for target age 50...
🤖 Simulating AI model processing...
🚀 Processing image from /content/image1.jpg for target age 50...
🤖 Simulating AI model processing...
🚀 Processing image from /content/image1.jpg for target age 50...
🤖 Simulating AI model processing...
🚀 Processing image from /content/image1.jpg for target age 50...
🤖 Simulating AI model processing...
🚀 Processing image from /content/image1.jpg for target age 50...
🤖 Simulating AI model proc

RecursionError: maximum recursion depth exceeded while calling a Python object

In [7]:
# Install libraries for the web server, real-time communication, and tunneling
!pip install fastapi "uvicorn[standard]" python-socketio pyngrok

# Install the library for Cloudinary integration
!pip install cloudinary

In [8]:
import cloudinary
import cloudinary.uploader
import cloudinary.api
from pyngrok import ngrok
import os

# --- ⚙️ Cloudinary Configuration ---
# Replace with your own Cloudinary credentials
cloudinary.config(
  cloud_name = "da52uzpu4",
  api_key = "545616752118824",
  api_secret = "fCn5NrmoD5-WpFVHa00q53Vsw7s",
  secure = True
)
print("✅ Cloudinary configured.")


# --- ⚙️ Ngrok Configuration ---
# Replace with your own Ngrok authtoken
NGROK_AUTH_TOKEN = "2z6GLE33Qw99xRpyn6hoATNvWSJ_6GUaHDoUQWUbdnqDgcW7D"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ Ngrok configured.")

✅ Cloudinary configured.
✅ Ngrok configured.


In [18]:
import time
import requests
from PIL import Image, ImageDraw, ImageFont

def run(image_url: str, target_age: int) -> str:
    """
    Placeholder for your AI model.
    This function downloads an image, simulates a long processing task,
    and returns the path to the generated output image.

    Args:
        image_url: The public URL of the user-uploaded image.
        target_age: The target age for the transformation.

    Returns:
        The local file path of the generated image.
    """
    print(f"🚀 Processing image from {image_url} for target age {target_age}...")

    # 1. Download the image from the provided URL
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        input_image = Image.open(response.raw).convert("RGB")
        # In your real code, you would save this to a temporary path
        # and pass that path to your alignment and model functions.
        input_path = "/content/temp_input.png"
        input_image.save(input_path)
    except Exception as e:
        print(f"❌ Failed to download image: {e}")
        raise

    # 2. **REPLACE THIS BLOCK WITH YOUR ACTUAL AI MODEL LOGIC**
    # This section simulates the time it takes for your model to run.
    print("🤖 Simulating AI model processing...")
    output_path = run_age_changer(input_path, target_age)
    time.sleep(10) # Simulate long processing time
    # Let's create a dummy output image for this example
    output_image = Image.open(output_path).convert("RGB")
    draw = ImageDraw.Draw(output_image)
    try:
        font = ImageFont.truetype("LiberationSans-Regular.ttf", 24)
    except IOError:
        font = ImageFont.load_default()
    draw.text((10, 10), f"Processed for age: {target_age}", fill="white", font=font)
    output_path = "/content/temp_output.png"
    output_image.save(output_path)
    # **END OF REPLACEMENT BLOCK**

    print(f"✅ AI processing complete. Output saved to {output_path}")
    return output_path

In [19]:
run("https://res.cloudinary.com/da52uzpu4/image/upload/v1754149566/gers8yezp5d3c2k2jrgf.png", 80)

🚀 Processing image from https://res.cloudinary.com/da52uzpu4/image/upload/v1754149566/gers8yezp5d3c2k2jrgf.png for target age 80...
🤖 Simulating AI model processing...
Aligned image has shape: (256, 256)
✅ AI processing complete. Output saved to /content/temp_output.png


'/content/temp_output.png'

In [20]:
import fastapi
import asyncio
import json
from fastapi import WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
from typing import Dict, Set

# Create a FastAPI app instance
app = fastapi.FastAPI()

# Allow all origins for CORS (Cross-Origin Resource Sharing) for easy development
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Store active WebSocket connections
active_connections: Dict[str, WebSocket] = {}

class ConnectionManager:
    def __init__(self):
        self.active_connections: Dict[str, WebSocket] = {}

    async def connect(self, websocket: WebSocket, client_id: str):
        await websocket.accept()
        self.active_connections[client_id] = websocket
        print(f"🤝 Client connected: {client_id}")

    def disconnect(self, client_id: str):
        if client_id in self.active_connections:
            del self.active_connections[client_id]
            print(f"👋 Client disconnected: {client_id}")

    async def send_message(self, message: dict, client_id: str):
        if client_id in self.active_connections:
            websocket = self.active_connections[client_id]
            await websocket.send_text(json.dumps(message))

manager = ConnectionManager()

@app.websocket("/ws/{client_id}")
async def websocket_endpoint(websocket: WebSocket, client_id: str):
    await manager.connect(websocket, client_id)

    try:
        while True:
            data = await websocket.receive_text()
            message = json.loads(data)

            # Handle different message types
            message_type = message.get('type')

            if message_type == 'process_image':
                await handle_process_image(client_id, message.get('data', {}))
            else:
                # Echo unknown message types back to client
                await manager.send_message({
                    'type': 'error',
                    'data': {'message': f'Unknown message type: {message_type}'}
                }, client_id)

    except WebSocketDisconnect:
        manager.disconnect(client_id)
    except Exception as e:
        print(f"❌ WebSocket error for {client_id}: {e}")
        manager.disconnect(client_id)

async def handle_process_image(client_id: str, data: dict):
    """
    Handles the image processing request from the client.
    """
    print(f"📩 Received processing request from {client_id}: {data}")
    image_url = data.get('image_url')
    target_age = data.get('target_age')

    if not image_url or target_age is None:
        await manager.send_message({
            'type': 'error',
            'data': {'message': 'Missing image_url or target_age.'}
        }, client_id)
        return

    try:
        # 1. Notify the client that processing has started
        await manager.send_message({
            'type': 'status',
            'data': {'message': 'Processing started... Model is running.'}
        }, client_id)

        # 2. Run the blocking AI function in a separate thread to avoid freezing the server
        output_file_path = await asyncio.to_thread(run, image_url, target_age)

        # 3. Upload the result to Cloudinary
        await manager.send_message({
            'type': 'status',
            'data': {'message': 'Uploading result...'}
        }, client_id)

        # Note: You'll need to import cloudinary at the top of your file
        # import cloudinary
        # import cloudinary.uploader
        upload_result = cloudinary.uploader.upload(output_file_path)
        final_url = upload_result.get('secure_url')

        # 4. Send the final image URL back to the client
        await manager.send_message({
            'type': 'result',
            'data': {'final_url': final_url}
        }, client_id)
        print(f"✅ Sent final URL to {client_id}: {final_url}")

    except Exception as e:
        print(f"❌ An error occurred: {e}")
        await manager.send_message({
            'type': 'error',
            'data': {'message': f"An error occurred: {str(e)}"}
        }, client_id)

print("✅ FastAPI app and WebSocket handlers are defined.")

✅ FastAPI app and WebSocket handlers are defined.


In [21]:
import uvicorn
import threading
import os

# Kill any existing ngrok processes
os.system("killall ngrok")
os.system("fuser -k 8000/tcp")

# Define the function to run the Uvicorn server
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Run the server in a separate thread so it doesn't block the notebook
server_thread = threading.Thread(target=run_server)
server_thread.start()

# Open a tunnel to the running server
public_url = ngrok.connect(8000)
print("==================================================================================")
print(f"✅ Your app is live!")
print(f"🔗 Public URL: {public_url}")
print(f"🔌 Your WebSocket endpoint for the frontend is: {public_url}/ws")
print("==================================================================================")

INFO:     Started server process [6492]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


✅ Your app is live!
🔗 Public URL: NgrokTunnel: "https://73a17ed8d43d.ngrok-free.app" -> "http://localhost:8000"
🔌 Your WebSocket endpoint for the frontend is: NgrokTunnel: "https://73a17ed8d43d.ngrok-free.app" -> "http://localhost:8000"/ws
